$$\textbf{Model Training}$$

In [ ]:
!huggingface-cli login
#Login with your huggingface credentials so you can upload your model to the hub

In [ ]:


import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,AutoModelForCausalLM,AutoConfig
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

df = pd.read_csv("/content/politcs_dataset.csv")
#All I really wanted was the title and the article and the combined text, adjust this to your dataset
df['title'] = df['title'].astype(str)
df['article'] = df['article'].astype(str)

df['combined'] = df['title'] + " " + df['article']
df = df.dropna()

dataset = Dataset.from_pandas(df)

model_name = "EleutherAI/gpt-neo-125m"
#Gpt neo utilizes the GPT2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)


#very simple training arguments
def tokenize_function(examples):
    tokens = tokenizer(examples["combined"], truncation=True)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset.save_to_disk("/content/tokenized_dataset")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/6630 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6630 [00:00<?, ? examples/s]

Quantifier


In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:


from accelerate import Accelerator
import os

training_args = TrainingArguments(
    output_dir="NewsGen",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Increase the batch size if you have more GPU memory
    per_device_eval_batch_size=1,  # Increase the batch size if you have more GPU memory
    gradient_accumulation_steps=8,  # Accumulate gradients over more steps
    num_train_epochs=6,  # Number of training epochs,  This was perfect for me to prevent overfitting and get a good model
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training; Disable this you don't have a GPU
    logging_dir='./logs',
    push_to_hub=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,

)

trainer.train()
